In [9]:
%load_ext autoreload
%autoreload 2
import risktools as rt
from pandas_datareader import data
import json
import os
import pandas as pd

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [12]:
def _load_json(fn, dataframe=True):
    # path = os.path.dirname(__file__)
    fp = os.path.join(fn)
    with open(fp) as js:
        if dataframe == True:
            df = pd.read_json(js)
            df.columns = df.columns.str.replace(".", "_")
        else:
            df = json.load(js)

    return df

In [35]:
# Test 1
ac = (
    _load_json("../pytest/data/returns1.json")
    .round(4)
    .set_index("date")
    .sort_index(axis=1)
)
ac.columns.name = "series"
ac = ac.loc['2004-01-05':'2022-01-05',:]

ts = (
    rt.returns(
        df=rt.data.open_data("dflong").round(
            4
        ),  # round(4) because R toJSON function does so
        ret_type="rel",
        period_return=1,
        spread=True,
    )
    .round(4)
    .sort_index(axis=1)
)
ts = ts.loc['2004-01-05':'2022-01-05',:]
ts = ts[ac.columns]

In [63]:
t = set(ts.dropna().index.strftime('%Y-%m-%d').unique())

In [64]:
a = set(ac.dropna().index.strftime('%Y-%m-%d').unique())

In [69]:
d = t.difference(a)

In [71]:
c = {element for element in a if element not in t}

In [73]:
print(c)

set()


In [76]:
print(list(d)[:10])

['2021-06-10', '2021-06-02', '2021-07-20', '2021-10-26', '2021-02-12', '2021-08-12', '2021-07-22', '2021-02-25', '2021-03-17', '2021-07-07']


In [79]:
ts.loc['2021-06-01':'2021-06-30', 'BRN01']

date
2021-06-01    0.0134
2021-06-02    0.0157
2021-06-03   -0.0006
2021-06-04    0.0081
2021-06-07   -0.0056
2021-06-08    0.0102
2021-06-09    0.0000
2021-06-10    0.0042
2021-06-11    0.0023
2021-06-14    0.0023
2021-06-15    0.0155
2021-06-16    0.0054
2021-06-17   -0.0176
2021-06-18    0.0059
2021-06-21    0.0189
2021-06-22   -0.0012
2021-06-23    0.0051
2021-06-24    0.0049
2021-06-25    0.0082
2021-06-28   -0.0197
2021-06-29    0.0011
2021-06-30    0.0049
Name: BRN01, dtype: float64

In [80]:
ac.loc['2021-06-01':'2021-06-30', 'BRN01']

date
2021-06-01    0.0134
2021-06-02    0.0157
2021-06-03   -0.0006
2021-06-04    0.0081
2021-06-07   -0.0056
2021-06-08    0.0102
2021-06-09    0.0000
2021-06-10    0.0042
2021-06-11    0.0023
2021-06-14    0.0023
2021-06-15    0.0155
2021-06-16    0.0054
2021-06-17   -0.0176
2021-06-18    0.0059
2021-06-21    0.0189
2021-06-22   -0.0012
2021-06-23    0.0051
2021-06-24    0.0049
2021-06-25    0.0082
2021-06-28   -0.0197
2021-06-29    0.0011
2021-06-30    0.0049
Name: BRN01, dtype: float64